In [50]:
import numpy as np
from torch.utils.data import DataLoader
from torch import nn
import random
import pygame
import math
import torch
from torch import optim
import time
import load
from models import EncoderRNN, DecoderRNN, NeighbourhoodFullyConnected, NeighbourhoodFullyConnectedDecoder
import load_topology_dataset
from load_topology_dataset import MAX_CELL_COUNT, MIN_CELL_COUNT, IMAGE_SIZE
device = "cpu"

# Simplifying Assumptions
1. No multiple carriers.
2. Locations of loads and cells discretized to grid of resolution IMAGE_SIZE
3. Cells are not limited by capacity
4. Distribution of load follows inverse square of distance.
5. Cells are omnidirectional.
6. Load is constant.
7. Only a maximum of 1 cell changes (addition or removal) at a time.

In [51]:
test_loads = ((1,12), (22, 22))
test_loads = [(1,12)]
test_cells = [(2, 12), (31, 0)]
load.calculate_cell_load(cells=test_cells, loads=test_loads)

[(2, 12, 0.6709275647328745), (31, 0, 0.3290724352671256)]

# Early Fusion

# Visualisation

In [40]:
PIXELS = 640
LEFT = 1
RIGHT = 3
INPUT_SCREEN_COUNT = 2
TARGET_SCREEN_COUNT = 2
SCREEN_COUNT = INPUT_SCREEN_COUNT + TARGET_SCREEN_COUNT

PIXELS_PER_BLOCK = int(PIXELS / IMAGE_SIZE)
SCALE_ARR = np.ones((PIXELS_PER_BLOCK, PIXELS_PER_BLOCK, 1))
print(PIXELS_PER_BLOCK)

load_screen = np.zeros((IMAGE_SIZE, IMAGE_SIZE, 1))
load_screen[1,12, 0] = 255
load_screen[22,22, 0] = 255
loads = [(1, 12), (22, 22)]
block_img = np.kron(load_screen, SCALE_ARR)

screen = pygame.display.set_mode((PIXELS, PIXELS))
screens = []

#for i in range(SCREEN_COUNT):
##    screen_draw = pygame.Surface((PIXELS, PIXELS))
#    screens.append(screen_draw)


screen_draw_idx = 0


draw_on = False
last_pos = (0, 0)
color = (255, 128, 0)
radius = 10
device = "cpu"

block_screen = np.zeros((IMAGE_SIZE, IMAGE_SIZE, 3))

model_screen = np.zeros((IMAGE_SIZE, IMAGE_SIZE, 3), dtype=np.uint8)
frame = 0


20


In [41]:
a = ["abc", "def", "ghi", "jkl"]

In [42]:
a[2:]

['ghi', 'jkl']

In [43]:
def add_cell_if_not_exists(new_cell, cells):
    cell_already_exists = False

    for i, cell in enumerate(cells):
        if cell[0] == new_cell[0] and cell[1] == new_cell[1]:
            cell_already_exists = True
            break
    if not cell_already_exists:
        cells.append(new_cell)
    return cells

def remove_cell_if_exists(remove_cell, cells):
    for i, cell in enumerate(cells):
        if cell[0] == remove_cell[0] and cell[1] == remove_cell[1]:
            cells.pop(i)
            break
    return cells

# Visualisation

In [48]:
screen_draw_idx = 0
cells_seq = [list() for i in range(SCREEN_COUNT)]
cells_load_seq = [list() for i in range(SCREEN_COUNT)]
BATCH_SIZE=1
show_labels = True

while True:
    
    ##############################
    # Use model to forecast future load
    ##############################
    load_cells_seq_input = cells_load_seq[:INPUT_SCREEN_COUNT]
    load_cells_seq_target = cells_load_seq[INPUT_SCREEN_COUNT:]
    
    model_target = [list() for i in range(TARGET_SCREEN_COUNT)]
    for reference_cell in cells_seq[-1]:
        #print(reference_cell)
        ref_x, ref_y = reference_cell[0], reference_cell[1]
        reference_cell_input = np.zeros((INPUT_SCREEN_COUNT))
        reference_cell_present_input = np.zeros((INPUT_SCREEN_COUNT))
        neighbourhood_cell_rel_input = np.zeros((INPUT_SCREEN_COUNT, MAX_CELL_COUNT, 2))
        neighbourhood_cell_load_input = np.zeros((INPUT_SCREEN_COUNT, MAX_CELL_COUNT))
        neighbourhood_cell_present_input = np.zeros((INPUT_SCREEN_COUNT, MAX_CELL_COUNT))
        for seq_idx in range(INPUT_SCREEN_COUNT):
            reference_cell_active = False
            cell_idx = 0
            for cell in load_cells_seq_input[seq_idx]:
                current_x = cell[0]
                current_y = cell[1]
                current_load = cell[2]
                # If not reference cell add to neighbourhood
                if not (current_x == ref_x and current_y == ref_y):
                    # neighbourhood normalized by dividing by IMAGE_SIZE.
                    # Subtract from 1 since cells that are closes have greated influence.
                    #print("seq_idx {}".format(seq_idx))
                    #print("cell_idx {}".format(cell_idx))
                    neighbourhood_cell_rel_input[seq_idx, cell_idx, 0] = 1. - (ref_x - current_x) / IMAGE_SIZE
                    neighbourhood_cell_rel_input[seq_idx, cell_idx, 1] = 1. - (ref_y - current_y) / IMAGE_SIZE
                    neighbourhood_cell_load_input[seq_idx, cell_idx] = current_load
                    cell_idx += 1
                else:
                    # print("ref cell")
                    reference_cell_input[seq_idx] = current_load
                    reference_cell_active = True
            if reference_cell_active:
                reference_cell_present_input[seq_idx] = 1.0

        load_cells_seq_target = cells_load_seq[INPUT_SCREEN_COUNT:]
        reference_cell_present_target = np.zeros((TARGET_SCREEN_COUNT))
        neighbourhood_cell_rel_target = np.zeros((TARGET_SCREEN_COUNT, MAX_CELL_COUNT, 2))


        for seq_idx in range(TARGET_SCREEN_COUNT):
            reference_cell_active = False
            cell_idx = 0
            for cell in load_cells_seq_target[seq_idx]:
                current_x = cell[0]
                current_y = cell[1]
                current_load = cell[2]
                #print(current_load)
                # If not reference cell add to neighbourhood
                if not (current_x == ref_x and current_y == ref_y):
                    # neighbourhood normalized by dividing by IMAGE_SIZE.
                    # Subtract from 1 since cells that are closes have greated influence.
                    neighbourhood_cell_rel_target[seq_idx, cell_idx, 0] = 1. - abs(ref_x - current_x) / IMAGE_SIZE
                    neighbourhood_cell_rel_target[seq_idx, cell_idx, 1] = 1. - abs(ref_y - current_y) / IMAGE_SIZE
                    cell_idx += 1
                else:
                    reference_cell_active = True
            if reference_cell_active:
                reference_cell_present_target[seq_idx] = 1.0

        reference_cell_input = torch.tensor(reference_cell_input, dtype=torch.float32).unsqueeze(0)
        reference_cell_present_input = torch.tensor(reference_cell_present_input, dtype=torch.float32).unsqueeze(0)
        neighbourhood_cell_rel_input = torch.tensor(neighbourhood_cell_rel_input, dtype=torch.float32).unsqueeze(0)
        neighbourhood_cell_load_input = torch.tensor(neighbourhood_cell_load_input, dtype=torch.float32).unsqueeze(0)
        reference_cell_present_target = torch.tensor(reference_cell_present_target, dtype=torch.float32).unsqueeze(0)
        neighbourhood_cell_rel_target = torch.tensor(neighbourhood_cell_rel_target, dtype=torch.float32).unsqueeze(0)

        for seq_idx in range(INPUT_SCREEN_COUNT):
            neighbourhood_influence = torch.zeros(BATCH_SIZE, 16)
            for neighbourhood_cell_idx in range(MAX_CELL_COUNT):
                output = neighbourhood_fully_connected(neighbourhood_cell_rel_input[:,seq_idx, neighbourhood_cell_idx, :].view(BATCH_SIZE, 2),
                                              neighbourhood_cell_load_input[:,seq_idx, neighbourhood_cell_idx].view(BATCH_SIZE, 1))
                neighbourhood_influence = torch.add(neighbourhood_influence, output)
            # Concatonate neighbourhood with reference cell time series.
            cat_ref_input = (torch.cat((reference_cell_input[:, seq_idx].view(BATCH_SIZE, 1),
                                                reference_cell_present_input[:, seq_idx].view(BATCH_SIZE, 1)), dim=1))
            encoder_input = torch.cat((cat_ref_input , neighbourhood_influence), dim=1).view(BATCH_SIZE, 1, 1 + 1+ 16)
            if seq_idx==0:
                encoder_hidden = torch.zeros(1, BATCH_SIZE, HIDDEN_SIZE, device=device)
            encoder_output, encoder_hidden = encoder(encoder_input, encoder_hidden)
        #print("encoder_hidden {}".format(encoder_hidden))

        for seq_idx in range(TARGET_SCREEN_COUNT):
            neighbourhood_influence = torch.zeros(BATCH_SIZE, 16)

            #print("reference_cell_input {}".format(reference_cell_input))
            
            for neighbourhood_cell_idx in range(MAX_CELL_COUNT):
                output = neighbourhood_fully_connected_decoder(neighbourhood_cell_rel_target[:,seq_idx, neighbourhood_cell_idx, :].view(BATCH_SIZE, 2))
                neighbourhood_influence = torch.add(neighbourhood_influence, output)

            #print("output {}".format(output))
            #print("neighbourhood_influence {}".format(neighbourhood_influence))
            if seq_idx==0:
                decoder_output, decoder_hidden = decoder(reference_cell_input[:, -1].view(BATCH_SIZE, 1),
                                                         reference_cell_present_input[:, -1].view(BATCH_SIZE, 1),
                                                         neighbourhood_influence.view(BATCH_SIZE, 16),
                                                         encoder_hidden)
                loss = criterion(decoder_output, reference_cell_target[:, seq_idx])

                decoder_output, decoder_hidden = decoder(decoder_output.view(BATCH_SIZE, 1),
                                                         reference_cell_present_target[:, seq_idx -1].view(BATCH_SIZE, 1),
                                                         neighbourhood_influence.view(BATCH_SIZE, 16),
                                                         decoder_hidden)
            model_target[seq_idx].append((ref_x, ref_y, decoder_output.detach().numpy()[0][0][0]))
            
        
        #print("decoder_target {}".format(decoder_output))
        #print("model_target {}".format(model_target))
    #print(decoder_output)
    ################################
    # DISPLAY
    #################################
    screen_draw = pygame.Surface((PIXELS, PIXELS))
    
    #print(cells_seq)
    cells = cells_seq[screen_draw_idx]
    cell_loads = cells_load_seq[screen_draw_idx]

    if (screen_draw_idx < INPUT_SCREEN_COUNT) or (show_labels and screen_draw_idx >= INPUT_SCREEN_COUNT):
        for cell in cell_loads:
            color = (cell[2] * 1024) - 256
            if color > 255:
                color = 255
            if color < 0:
                color = 0
            pygame.draw.rect(
            screen_draw,
            (color, 50, 0),
            (
                cell[0] * PIXELS_PER_BLOCK,
                cell[1] * PIXELS_PER_BLOCK,
                PIXELS_PER_BLOCK,
                PIXELS_PER_BLOCK,
            ),
            )
    else:
        model_loads = model_target[screen_draw_idx - INPUT_SCREEN_COUNT]
        for cell in model_loads:
            color = (cell[2] * 1024) - 256
            if color > 255:
                color = 255
            if color < 0:
                color = 0
            pygame.draw.rect(
            screen_draw,
            (color, 50, 0),
            (
                cell[0] * PIXELS_PER_BLOCK,
                cell[1] * PIXELS_PER_BLOCK,
                PIXELS_PER_BLOCK,
                PIXELS_PER_BLOCK,
            ),
            )

    e = pygame.event.poll()


    if e.type == pygame.QUIT:
        raise StopIteration
    if e.type == pygame.MOUSEBUTTONDOWN:
        leftclick, middleclick, rightclick = pygame.mouse.get_pressed()
        x, y = e.pos
        x_cell= int(x / PIXELS_PER_BLOCK)
        y_cell = int(y / PIXELS_PER_BLOCK)
        if leftclick:
            cells = add_cell_if_not_exists((x_cell, y_cell), cells)
        elif rightclick:
            cells = remove_cell_if_exists((x_cell, y_cell), cells)            
        draw_on = True
        # Update cells
        cells_seq[screen_draw_idx] = cells
        
        # Update all cell loads
        for i, cells in enumerate(cells_seq):
            cells_load_seq[i] = load.calculate_cell_load(cells, loads)
    
    if e.type == pygame.KEYDOWN:
        if e.key == pygame.K_SPACE:
            if show_labels:
                show_labels = False
            else:
                show_labels = True
                print(model_target[-1])
                print(load_cells_seq_target[-1])
        if e.key ==  pygame.K_LEFT:
            if screen_draw_idx > 0:
                screen_draw_idx -= 1
        if e.key ==  pygame.K_RIGHT:
            if screen_draw_idx < SCREEN_COUNT - 1:
                screen_draw_idx += 1
    if e.type == pygame.MOUSEBUTTONUP:
        draw_on = False
    pygame.display.flip()
    target_screen = pygame.surfarray.array3d(screen_draw)
    display_img = block_img + target_screen
    new_surf = pygame.pixelcopy.make_surface(display_img.astype(np.uint8))
    screen.blit(new_surf, (0, 0))

[(12, 13, 2.1651034), (31, 30, 0.51882005), (30, 29, 0.51775223), (29, 28, 0.5157211), (28, 27, 0.51380646)]
[(12, 13, 0.52716680413611), (31, 30, 0.33990757386327436), (30, 29, 0.35951307994079995), (29, 28, 0.37801105200473534), (28, 27, 0.3954014900550805)]
[(12, 13, 2.1651034), (31, 30, 0.51882005), (30, 29, 0.51775223), (29, 28, 0.5157211), (28, 27, 0.51380646)]
[(12, 13, 0.52716680413611), (31, 30, 0.33990757386327436), (30, 29, 0.35951307994079995), (29, 28, 0.37801105200473534), (28, 27, 0.3954014900550805)]
[(12, 13, 2.1651034), (31, 30, 0.51882005), (30, 29, 0.51775223), (29, 28, 0.5157211), (28, 27, 0.51380646), (14, 14, 0.4777406)]
[(12, 13, 0.41322526192672404), (31, 30, 0.2721889121335957), (30, 29, 0.2872491090183028), (29, 28, 0.3014405157593933), (28, 27, 0.31476313235686726), (14, 14, 0.4111330688051169)]
[(12, 13, 2.1651034), (31, 30, 0.51882005), (30, 29, 0.51775223), (29, 28, 0.5157211), (28, 27, 0.51380646), (14, 14, 0.4777406)]
[(12, 13, 0.41322526192672404), (31

KeyboardInterrupt: 

# Training

In [52]:
INPUT_SEQ_LEN = 5
TARGET_SEQ_LEN = 5
HIDDEN_SIZE = 32
# TODO fix BATCH_SIZE > 1 is bugged.
BATCH_SIZE = 2
TEACHER_FORCING_PROB = 0.5
LEARNING_RATE = 1e-3
PRINT_LOSS_EVERY = 1000
neighbourhood_fully_connected = NeighbourhoodFullyConnected(2, 1, 16, 16)
neighbourhood_fully_connected_decoder = NeighbourhoodFullyConnectedDecoder(2, 16, 16)
encoder = EncoderRNN(1 + 1 + 16, HIDDEN_SIZE)
decoder = DecoderRNN(18, HIDDEN_SIZE)
cell_load_dataset = load_topology_dataset.LoadCellDataset(initial_cell_counts=(2, 5), initial_load_counts=(2,5),
                                    input_seq_len=INPUT_SEQ_LEN, target_seq_len=TARGET_SEQ_LEN, network_mutate_prob=[0.33, 0.66
                                                                                                                    ])
dataloader = DataLoader(cell_load_dataset, batch_size=BATCH_SIZE,
                        shuffle=False, num_workers=1)

parameters = (list(neighbourhood_fully_connected.parameters()) + 
                       list(neighbourhood_fully_connected_decoder.parameters()) +
                       list(encoder.parameters()) +
                       list(decoder.parameters()))

optimizer = optim.Adam(parameters
                       , lr=LEARNING_RATE)
criterion = nn.MSELoss()
avg_loss_window = np.zeros(PRINT_LOSS_EVERY)
avg_loss_window_idx = 0

for batch_idx, data in enumerate(dataloader):
    reference_cell_input =  data[0]
    reference_cell_present_input = data[1]
    neighbourhood_cell_rel_input = data[2]
    neighbourhood_cell_load_input = data[3]
    reference_cell_target = data[4]
    reference_cell_present_target = data[5]
    neighbourhood_cell_rel_target = data[6]
    optimizer.zero_grad()
    for seq_idx in range(reference_cell_input.size(1)):
        neighbourhood_influence = torch.zeros(BATCH_SIZE, 16)
        for neighbourhood_cell_idx in range(MAX_CELL_COUNT):
            output = neighbourhood_fully_connected(neighbourhood_cell_rel_input[:,seq_idx, neighbourhood_cell_idx, :].view(BATCH_SIZE, 2),
                                          neighbourhood_cell_load_input[:,seq_idx, neighbourhood_cell_idx].view(BATCH_SIZE, 1))
            neighbourhood_influence = torch.add(neighbourhood_influence, output)
        # Concatonate neighbourhood with reference cell time series.
        cat_ref_input = (torch.cat((reference_cell_input[:, seq_idx].view(BATCH_SIZE, 1),
                                            reference_cell_present_input[:, seq_idx].view(BATCH_SIZE, 1)), dim=1))
        encoder_input = torch.cat((cat_ref_input , neighbourhood_influence), dim=1).view(BATCH_SIZE, 1, 1 + 1+ 16)
        if seq_idx==0:
            encoder_hidden = torch.zeros(1, BATCH_SIZE, HIDDEN_SIZE, device=device)
        encoder_output, encoder_hidden = encoder(encoder_input, encoder_hidden)
    
    for seq_idx in range(reference_cell_target.size(1)):
        neighbourhood_influence = torch.zeros(BATCH_SIZE, 16)

        for neighbourhood_cell_idx in range(MAX_CELL_COUNT):
            output = neighbourhood_fully_connected_decoder(neighbourhood_cell_rel_target[:,seq_idx, neighbourhood_cell_idx, :].view(BATCH_SIZE, 2))
            neighbourhood_influence = torch.add(neighbourhood_influence, output)

        if seq_idx==0:
            decoder_output, decoder_hidden = decoder(reference_cell_input[:, -1].view(BATCH_SIZE, 1),
                                                     reference_cell_present_input[:, -1].view(BATCH_SIZE, 1),
                                                     neighbourhood_influence.view(BATCH_SIZE, 16),
                                                     encoder_hidden)
            loss = criterion(decoder_output, reference_cell_target[:, seq_idx])
        if random.random() > TEACHER_FORCING_PROB:
            decoder_output, decoder_hidden = decoder(reference_cell_target[:, seq_idx - 1].view(BATCH_SIZE, 1),
                                                     reference_cell_present_target[:, seq_idx -1].view(BATCH_SIZE, 1),
                                                     neighbourhood_influence.view(BATCH_SIZE, 16),
                                                     decoder_hidden)
        else:
            decoder_output, decoder_hidden = decoder(decoder_output.view(BATCH_SIZE, 1),
                                                     reference_cell_present_target[:, seq_idx -1].view(BATCH_SIZE, 1),
                                                     neighbourhood_influence.view(BATCH_SIZE, 16),
                                                     decoder_hidden)
            loss += criterion(decoder_output, reference_cell_target[:, seq_idx])

            
    loss.backward()
    avg_loss_window[avg_loss_window_idx] = loss.data
    if avg_loss_window_idx >= PRINT_LOSS_EVERY - 1:
        avg_loss_window_idx = 0
        print(avg_loss_window.mean())
        #for param in parameters:
            #print(param.shape)
            #print(param.grad.data)
        print(decoder_output - reference_cell_target[:, seq_idx])
    else:
        avg_loss_window_idx += 1         
    optimizer.step()

3.525749309424311
tensor([[[ 0.4217,  0.1505]],

        [[ 0.0130, -0.2582]]], grad_fn=<SubBackward0>)
3.6756270584464072
tensor([[[-0.6696, -0.1308]],

        [[-0.7274, -0.1885]]], grad_fn=<SubBackward0>)
3.106605948504992
tensor([[[-0.1177, -4.3496]],

        [[ 1.4640, -2.7679]]], grad_fn=<SubBackward0>)
3.338417785162106
tensor([[[-2.2205,  2.7795]],

        [[-4.7280,  0.2720]]], grad_fn=<SubBackward0>)


KeyboardInterrupt: 

# Legacy Visualisation

In [12]:
screen_draw_idx = 0
cells_seq = [list() for i in range(SCREEN_COUNT)]

def add_cell_if_not_exists(new_cell, cells):
    cell_already_exists = False

    for i, cell in enumerate(cells):
        if cell[0] == new_cell[0] and cell[1] == new_cell[1]:
            cell_already_exists = True
            break
    if not cell_already_exists:
        cells.append(new_cell)
    return cells

def remove_cell_if_exists(remove_cell, cells):
    for i, cell in enumerate(cells):
        if cell[0] == remove_cell[0] and cell[1] == remove_cell[1]:
            cells.pop(i)
            break
    return cells

while True:
    
    
    screen_draw = pygame.Surface((PIXELS, PIXELS))
    
    #print(cells_seq)
    cells = cells_seq[screen_draw_idx]
    if len(cells) > 0:
        cell_loads = load.calculate_cell_load(cells, loads)
        #print(cell_loads)
        #print(loads)



        # Update screen
        for cell in cell_loads:
            color = (cell[2] * 1024) - 256
            if color > 255:
                color = 255
            if color < 0:
                color = 0
            pygame.draw.rect(
            screen_draw,
            (color, 50, 0),
            (
                cell[0] * PIXELS_PER_BLOCK,
                cell[1] * PIXELS_PER_BLOCK,
                PIXELS_PER_BLOCK,
                PIXELS_PER_BLOCK,
            ),
            )
        

    e = pygame.event.poll()


    if e.type == pygame.QUIT:
        raise StopIteration
    if e.type == pygame.MOUSEBUTTONDOWN:
        leftclick, middleclick, rightclick = pygame.mouse.get_pressed()
        x, y = e.pos
        x_cell= int(x / PIXELS_PER_BLOCK)
        y_cell = int(y / PIXELS_PER_BLOCK)
        if leftclick:
            cells = add_cell_if_not_exists((x_cell, y_cell), cells)
        elif rightclick:
            cells = remove_cell_if_exists((x_cell, y_cell), cells)            
        draw_on = True
        # Update cells
        cells_seq[screen_draw_idx] = cells
    
    if e.type == pygame.KEYDOWN:
        if e.key == pygame.K_SPACE:
            if freeze:
                freeze = False
            else:
                freeze = True
        if e.key ==  pygame.K_LEFT:
            if screen_draw_idx > 0:
                screen_draw_idx -= 1
        if e.key ==  pygame.K_RIGHT:
            if screen_draw_idx < SCREEN_COUNT - 1:
                screen_draw_idx += 1
    if e.type == pygame.MOUSEBUTTONUP:
        draw_on = False
    pygame.display.flip()
    target_screen = pygame.surfarray.array3d(screen_draw)
    display_img = block_img + target_screen
    new_surf = pygame.pixelcopy.make_surface(display_img.astype(np.uint8))
    screen.blit(new_surf, (0, 0))

KeyboardInterrupt: 